In [1]:
library(PQLseq)

In [2]:
fam = read.table('../gemma/plink_files/16p12_RNA_by_rep.fam', sep=' ')

In [3]:
pheno = read.table('../16p12.2_rnaseq_analysis/data/pheno_final.tsv', 
                   sep='\t', header=TRUE, stringsAsFactors = F)
rownames(pheno) = pheno$samp

In [4]:
counts = read.table('../16p12.2_rnaseq_analysis/data/16p12_lcl_gene_reads_underscores.gct',
                   sep='\t', header=T)
rownames(counts) = counts$Name
counts$Name = NULL

gencode2ensembl = function(s) {
    return(strsplit(s, '.', fixed=T)[[1]][1])
}
simple_sample = function(s) {
    sn = strsplit(s, '_', fixed=T)[[1]]
    sn = sn[length(sn)]
    return(sn)
}

rownames(counts) = unlist(lapply(rownames(counts), gencode2ensembl))
colnames(counts) = unlist(lapply(colnames(counts), simple_sample))

In [5]:
kinship = read.table('../gemma/output/relatedness_matrix_by_replicate.cXX.txt', sep='\t')
colnames(kinship) = fam$V2
rownames(kinship) = fam$V2

In [7]:
samps = colnames(counts)

pheno = pheno[samps,]
counts = counts[, samps]
kinship = kinship[samps, samps]

status = pheno$status3
status = as.numeric(as.factor(status))
status = 2 - status

In [9]:
fit = pqlseq(RawCountDataSet=counts, 
             Phenotypes=status, 
             RelatednessMatrix=kinship,
             fit.model="PMM")

## number of total individuals:  96 
## number of total genes/sites:  38062 
## number of adjusted covariates:  0 


Warning message in pqlseq(RawCountDataSet = counts, Phenotypes = status, RelatednessMatrix = kinship, :
“PQLseq::the relatedness matrix is singular, it has been modified!”


ERROR: Error in isOpen(con): invalid connection


In [11]:
write.table(fit, 'pqlseq_by_replicate.tsv', sep='\t')